<a href="https://colab.research.google.com/github/mAcvz/Stage/blob/main/nvtg2_sample(3_indici).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Create folder and intialization files

In [1]:
%%bash 
mkdir RUN

In [2]:
%%bash
touch conf.xyz

### my_calcener_file

In [5]:
from numpy import zeros, rint, sqrt, sum, pi, int64, float64
from numpy import int32
import numpy 
from numba import cuda
from math import ceil
#import cupy as cp
        
def calcener( mx, my, mz, N, rx, ry, rz, Lx, Ly, Lz, sp, r2cut, c12, c6, ecut, cf12, cf6, fx, fy, fz, etxx, stxx, styy, stzz, stxy, stxz, styz) :
    # zeroing
    # nb: rectangular PBC
    ene=0.
    vip=0.
    # arrey (numpy) temporanei 
    t_e1xx = zeros(N) 
    t_s1xx = zeros(N)
    t_s1yy = zeros(N)
    t_s1zz = zeros(N)
    t_s1xy = zeros(N)
    t_s1xz = zeros(N)
    t_s1yz = zeros(N)
    t_f1x  = zeros(N)
    t_f1y  = zeros(N)
    t_f1z  = zeros(N)
    # posizioni  assolute delle particelle
    rcx = zeros(N)
    rcy = zeros(N)
    rcz = zeros(N)
    # numero di celle in cui è stata divisa la box
    ncells=mx*my*mz
    # np conterrà il numero di particelle nella cella della particella considerata
    np   = zeros(ncells, dtype=int32)
    # l'indice indc contiene l'indice della cella in cui si trova l'i-esima particella (mi dirà dove si trova ogni particella)
    indc = zeros(N, dtype=int32)
    # tiene traccia della specie atomica 
    s1p  = zeros(N, dtype=int32)
    # generare l'indice della cella c-esima
    for i in range(N):
        vcx=int(mx*(rx[i]+0.5))
        vcy=int(my*(ry[i]+0.5)) 
        vcz=int(mz*(rz[i]+0.5))
        # cell index
        c = mz*(my*vcx+vcy)+vcz
        # ad ogni particella associo la sua cella di appartenenza
        indc[i]=c
        # vado ad accumulare il numero di particelle dentro ongi cella c 
        np[c] += 1
    # ogni posizione corrisonde ad una cella e contiene il numero di particelle conenute dalla calle 1 alla cella i-esima
    indp = zeros(ncells+1, dtype=int32)
    for c in range(0,ncells) :
        # ad ogni itezione sommo alla quantità precedente il numero di particelle contenute nella cella considerata
        indp[c+1] = indp[c] + np[c]
    # vettore che servirà a riordinare le particelle 
    indcp= zeros(N, dtype=int32)
    for i in range(N):
        '''
            la coppia (i,c) rapresenta la i-esima particella e la sua cella di
            appartenenza. 
            Si usa indp[c] += 1 in modo da visitare tutte le particelle conenute
            nella c-esima box. 
            ## rx,ry,rz sono le posizioni relative definite come segue:
                self.rx  =  self.x/self.Lx
                self.rx -= rint(self.rx)
        '''
        c=indc[i] # c èl'indice cella di appartenenza della particella i-esima considerata
        # passiamo dalle posizioni relaitve a quelle assoulute
        rcx[indp[c]] = (rx[i]+0.5)*Lx #  0 < rcx < Lx
        rcy[indp[c]] = (ry[i]+0.5)*Ly #  0 < rcy < Ly
        rcz[indp[c]] = (rz[i]+0.5)*Lz #  0 < rcz < Lz
        s1p[indp[c]] = sp[i]          # specie atomica della partcielle i-esima
        indcp[indp[c]] = i            # ora "indcp" contiene tutte le particelle ordinate per cella di appartenenza
        indp[c] += 1 # aggiungo 1 in modo da visitare tutte le particelel contenute nella cella c-esima 
    #
    # need to reconstruct index list
    # infatti per visitare tutte le particelle abbiamo eseguito indp[c] += 1 e abbiamo alterato indp
    indp[0]=0
    for c in range(0,ncells) :
        indp[c+1] = indp[c] + np[c]
    # indexing neighbour cells of selected one
    #
    ncmax = int(numpy.amax(np)) # numero massimo di particelle contenuto in un sola cella 
    nter = int(ncmax*27) # stima per eccesso il numero di particelle contenuto nelle 27 (blocco 3x3) celle vicine 
    vcx1 = zeros(27, dtype=int32) # vettore di numeri interi usati per riscostruire l'indice c1 della cella 
    vcy1 = zeros(27, dtype=int32) # vettore di numeri interi usati per riscostruire l'indice c1 della cella 
    vcz1 = zeros(27, dtype=int32) # vettore di numeri interi usati per riscostruire l'indice c1 della cella 
    #
    rci = zeros((ncmax,mz,3)) # permette di tenere traccia della posizione (x,y,z) di ogni particella in una particoalre cella "c" (lungo la direzone z)
    rcn = zeros((nter,mz,3))  # permette di tenere traccia della posizione (x,y,z) di ogni particella in una delle 27 celle (lungo la direzone z) 
    s1pi = zeros((ncmax,mz))  # permette di tenere traccia della specie atomica di ogni particella in una particoalre cella "c" (lungo la direzone z)
    s1pn = zeros((nter,mz))   # permette di tenere traccia della specie atomica di ogni particella in una delle 27 celle (lungo la direzone z) 
    #
    npi = zeros(mz, dtype=int32) #  per ogni cella conta il numero di particelle contenute (muovendosi lungo z)
    npj = zeros(mz,dtype=int32)  #  per ogni blocco 3x3 conta il numero di particelle contenute (muovendosi lungo z)
    # vcx,vcy,vcz: interi necessari a calcolare la posizioni delle celle vicine nel cubo 3x3
    k=0
    for i in range(-1,2):
        for j in range(-1,2):
            for l in range(-1,2):
                vcx1[k]= i
                vcy1[k]= j
                vcz1[k]= l
                k+=1
    # Loop over Cells
    for vcx in range(mx):
        for vcy in range(my):
            # arrays temporanei che conterrano le grandezze volute relative alle particelle di una cella (muovendosi lungo z)
            c_e1xx = zeros((ncmax,mz,nter))
            c_s1xx = zeros((ncmax,mz,nter))
            c_s1yy = zeros((ncmax,mz,nter))
            c_s1zz = zeros((ncmax,mz,nter))
            c_s1xy = zeros((ncmax,mz,nter))
            c_s1xz = zeros((ncmax,mz,nter))
            c_s1yz = zeros((ncmax,mz,nter))
            c_f1x  = zeros((ncmax,mz,nter))
            c_f1y  = zeros((ncmax,mz,nter))
            c_f1z  = zeros((ncmax,mz,nter))
            # scorro le celle lungo la direzione z     
            for vcz in range(mz):
                # creo l'indice della cella c-esima
                c = mz*(my*vcx+vcy)+vcz
                # loop over particles inside selected cells (central+neighbours)
                conti = 0
                for i in range(indp[c],indp[c+1]):
                    rci[conti,vcz,0] = rcx[i] # immagazzino le posizioni lungo x delle particelle nella cella c 
                    rci[conti,vcz,1] = rcy[i] # immagazzino le posizioni lungo y delle particelle nella cella c 
                    rci[conti,vcz,2] = rcz[i] # immagazzino le posizioni lungo z delle particelle nella cella c 
                    s1pi[conti,vcz] = s1p[i]  # tengo traccia del tipo di particella nella cella "c"
                    conti += 1
                npi[vcz] = conti  # tengo traccia del numero di particelle contenute nella cella "c"
                contj = 0         # tiene traccia del numero di particelle contenute nelle 27 celle vicine  
                # indice "c1" di una delle 27 celle vicine
                for k in range(27) :
                    wcx=vcx + vcx1[k]
                    wcy=vcy + vcy1[k]
                    wcz=vcz + vcz1[k]
                    # Periodic boundary conditions 
                    shiftx = 0.
                    if (wcx == -1) :
                        shiftx =-Lx
                        wcx = mx-1
                    elif (wcx==mx) :
                        shiftx = Lx
                        wcx = 0
                    shifty = 0.
                    if (wcy == -1) :
                        shifty =-Ly
                        wcy = my-1
                    elif (wcy==my) :
                        shifty = Ly
                        wcy = 0
                    shiftz = 0.
                    if (wcz == -1) :
                        shiftz =-Lz
                        wcz = mz-1
                    elif (wcz==mz) :
                        shiftz = Lz
                        wcz = 0
                    # determino l'indice della cella prima viciana c1
                    c1 = mz*(my*wcx+wcy)+wcz
                    # eseguimao un loop sulle particelle della cella c1 e salvo le posizioni delle particelle in "c1"
                    for j in range(indp[c1],indp[c1+1]):
                        rcn[contj,vcz,0] = rcx[j]+shiftx # salvo le posizioni lungo x delle particelle in "c1"
                        rcn[contj,vcz,1] = rcy[j]+shifty # salvo le posizioni lungo y delle particelle in "c1"
                        rcn[contj,vcz,2] = rcz[j]+shiftz # salvo le posizioni lungo z delle particelle in "c1"
                        s1pn[contj,vcz] = s1p[j] # tengo traccia del tipo di particella nella cella "c1"
                        contj += 1
                # il ciclo è stato ripetuto 27 volte quindi contj = numero di particelle nelle 27 celle 
                npj[vcz] = contj # tengo traccia del numero di particelle contenute nelle celle prime vicine a c
            #
            # CHIAMATA KERNEL
            threadsperblock = (8,8,8) # attezione non tutti i valori sono ammessi ??? come mai ??? 
            blockspergrid_x = ceil(rci.shape[0] / threadsperblock[0])
            blockspergrid_y = ceil(npj.shape[0] / threadsperblock[1])
            blockspergrid_z = ceil(nter / threadsperblock[2])
            blockspergrid = (blockspergrid_x, blockspergrid_y,blockspergrid_z)
            calgpu[blockspergrid, threadsperblock](rci,rcn,npi,npj,r2cut,s1pi,s1pn,c12,c6,cf12,cf6,ecut,c_f1x,c_f1y,c_f1z,c_s1xx,c_s1yy,c_s1zz,c_s1xy,c_s1xz,c_s1yz,c_e1xx)
            #
            # trasferire  su array che contengano tutte le N particelle & sommando sul terzo indice 
            for vcz in range(mz) :
                c = mz*(my*vcx+vcy)+vcz
                for i in range(npi[vcz]):
                    # genero nuovamente l'indice della cella 
                    ii =  indcp[indp[c] + i]
                    fx[ii]  = sum(c_f1x[i,vcz ,:])
                    fy[ii]  = sum(c_f1y[i,vcz ,:])
                    fz[ii]  = sum(c_f1z[i,vcz ,:])
                    etxx[ii] = sum(c_e1xx[i,vcz,:])
                    stxx[ii] = sum(c_s1xx[i,vcz,:])
                    styy[ii] = sum(c_s1yy[i,vcz,:])
                    stzz[ii] = sum(c_s1zz[i,vcz,:])
                    stxy[ii] = sum(c_s1xy[i,vcz,:])
                    stxz[ii] = sum(c_s1xz[i,vcz,:])
                    styz[ii] = sum(c_s1yz[i,vcz,:])

    ene = sum(etxx[:])
    vip = sum(stxx[:]+styy[:]+stzz[:])
    
    return ( 0.5*ene, 0.5*vip )

@cuda.jit
def calgpu(rci,rcn,npi,npj,r2cut,s1pi,ns1p,c12,c6,cf12,cf6,ecut,f1x,f1y,f1z,s1xx,s1yy,s1zz,s1xy,s1xz,s1yz,e1xx):
    # 
    # rci = zeros((ncmax,mz,3)) # permette di tenere traccia della posizione (x,y,z) di ogni particella in una particoalre cella c lungo la direzone z 
    # rcn = zeros((nter,mz,3))  # permette di tenere traccia della posizione (x,y,z) di ogni particella in una particoalre cella c1 lungo la direzone z 
    # s1pi = zeros((nter,mz)) # inerente alla specie atomica 
    # s1pn = zeros((nter,mz)) # inerente alla specie atomica 
    # npi = zeros(mz) # npi per ogni cella lungo z conta il numero di particelle contenute
    # npj = zeros(mz) # conta il numero di particelle contenute in tutte 27 le celle vicine 
    #
    # pos 1 scorre sulle particelle  all'interno di una cella 
    # pos 2 scorre sulle celle lungo z con x,y fissate
    # pos 3 scorre su tutte le particelle contenute nelle 27 celle vicine
    #
    pos1,pos2,pos3 = cuda.grid(3)
    #
    if pos2 < npj.shape[0]:        # pos2 deve sempre essere minore di mz ovvero npj.shape[0]
        if pos1 < npi[pos2]:       # pos1 deve sempre essere minore del numero di particelle contenute nella cella considerata
            if pos3 < npj[pos2]:   # pos3 deve sempre essere minore del numero di particelle contenute nelle 27  celle vicine 
                dx = rci[pos1,pos2,0] - rcn[pos3,pos2,0] 
                dy = rci[pos1,pos2,1] - rcn[pos3,pos2,1]
                dz = rci[pos1,pos2,2] - rcn[pos3,pos2,2]
                r2 = dx*dx + dy*dy + dz*dz
                if (r2<r2cut and r2>0.1) :
                    rr2 = 1./r2
                    rr6 = rr2*rr2*rr2
                    index = int(s1pi[pos1,pos2] + ns1p[pos3,pos2])
                    eij = (c12[index]*rr6 - c6[index])*rr6 + ecut[index]
                    vij = (cf12[index]*rr6 - cf6[index])*rr6 
                    vij *= rr2
                    
                    e1xx[pos1,pos2,pos3] = eij
                    f1x[pos1,pos2,pos3]  = vij * dx
                    f1y[pos1,pos2,pos3]  = vij * dy
                    f1z[pos1,pos2,pos3]  = vij * dz 
                    s1xx[pos1,pos2,pos3] = vij * dx * dx
                    s1yy[pos1,pos2,pos3] = vij * dy * dy 
                    s1zz[pos1,pos2,pos3] = vij * dz * dz 
                    s1xy[pos1,pos2,pos3] = vij * dx * dy 
                    s1xz[pos1,pos2,pos3] = vij * dx * dz
                    s1yz[pos1,pos2,pos3] = vij * dy * dz

                    


### nvtg_lj

In [6]:
#from numba import jitclass
#from numba import int32,int64,float64
from numpy import zeros, rint, sqrt, sum, pi, int64, float64
from numpy import random
from numpy import int32
from numpy import savetxt, column_stack, empty, str_
from pickle import dump, load, HIGHEST_PROTOCOL
#from my_calcener_file import calcener
from numba import njit 


                                            
  
class LJ :

  def __init__(self, rho, mx, my, mz, gforce, nstep ):
    #-start-------------------------
    self.mb   = 1.   # reference Argon 39.95 amu
    epsb      = 1.   # reference Argon 119 K
    sigb      = 1.   # reference Argon 0.3405 nm
    self.ma   = 1.   # Xe 131.29 amu = 3.29 m_Ar   # Kr 83.80 amu = 2.1 m_Ar
    epsa      = 1.   # Xe 204 K = 1.72 eps_Ar      # Kr 165 K = 1.39 eps_Ar
    siga      = 1.   # Xe 0.3975 nm =1.17 sigma_Ar # Kr 0.3633 nm = 1.07 sigma_Ar
    self.gforce = gforce
    self.c6   = zeros(3)
    self.cf6  = zeros(3)
    self.c12  = zeros(3)
    self.cf12 = zeros(3)
    self.ecut = zeros(3)
    # potential cut-off exchange order as needed 
    # WCA cut-off in the minimum
    # self.r2cut=2.**(1./3.)*max(sig1,sig2)**2
    # standard cut-off for LJ systems
    #self.rcut = max(siga,sigb)
    #self.rcut*= 3.
    # rcut needs to be an integer number of unit cell a
    a = (4/rho)**(1./3.)
    self.dslice = 2  
    cell_lenght = self.dslice*a
    self.rcut = 3.*sigb
    if (cell_lenght < self.rcut):
        self.rcut = cell_lenght
    self.lb = mz*self.dslice
    self.l1  =  mx*self.dslice
    self.l2  =  my*self.dslice ### add -2 to assume it is enough to have one lower and one higher extra box
    self.l3  =  mz*self.dslice
    self.npart   = 4*self.l1*self.l2*self.l3
    self.Lx = mx * cell_lenght
    self.Ly = my * cell_lenght
    self.Lz = mz * cell_lenght
    self.rho = self.npart/(mx*(my-2)*mz*cell_lenght**3)    
    ndim = self.npart
    # N initial guess of average number of particles for dimensioning
    self.x       = zeros( ndim )
    self.y       = zeros( ndim )
    self.z       = zeros( ndim )
    self.m       = zeros( ndim )
    self.sp      = zeros( ndim, dtype=int64 )
    self.rx      = zeros( ndim )
    self.ry      = zeros( ndim )
    self.rz      = zeros( ndim )
    self.px      = zeros( ndim )
    self.py      = zeros( ndim )
    self.pz      = zeros( ndim )
    self.fx      = zeros( ndim )
    self.fy      = zeros( ndim )
    self.fz      = zeros( ndim )
    self.etxx    = zeros( ndim )
    self.stxx    = zeros( ndim )
    self.styy    = zeros( ndim )
    self.stzz    = zeros( ndim )
    self.stxy    = zeros( ndim )
    self.stxz    = zeros( ndim )
    self.styz    = zeros( ndim )
    # dimensioning array for nemd
    self.heatt = zeros( (      2,nstep+1), dtype=float64 )
    self.hstat = zeros( (self.lb,nstep+1), dtype=float64 )
    self.hstbt = zeros( (self.lb,nstep+1), dtype=float64 )
    self.enkat = zeros( (self.lb,nstep+1), dtype=float64 )
    self.enkbt = zeros( (self.lb,nstep+1), dtype=float64 )
    self.jmazt = zeros( (self.lb,nstep+1), dtype=float64 )
    self.gzt   = zeros( (self.lb,nstep+1), dtype=float64 )
    self.jezt  = zeros( (self.lb,nstep+1), dtype=float64 )
    self.enet  = zeros( (self.lb,nstep+1), dtype=float64 )
    #
    self.kg      = 512
    self.gcount  = zeros( (self.kg,3) )
    self.ekin    = 0.0
    self.ene     = 0.0
    self.etot    = 0.0
    #
    self.tt      = 0.0
    self.ekt     = 0.0
    self.ept     = 0.0
    self.pres    = 0.0
    #
    rmax = min( (self.Lx, self.Ly, self.Lz) )/2.
    rmax = self.rcut # to compute gdr in calcener
    self.ldel  = rmax/self.kg
    self.r2max = rmax * rmax
    self.r2cut = self.rcut**2
    # particles N = Na + Nb
    N  = self.npart
    Na = self.npart//2
    # particle species: a
    self.sp[0:Na]= 0
    self.m[0:Na] = self.ma
    # particle species: b
    self.sp[Na:N]= 1
    self.m[Na:N] = self.mb
    # a-a interaction
    sig6        = siga**6
    self.c6[0]  = 4.0*epsa*sig6;
    self.c12[0] = self.c6[0]*sig6;
    self.cf12[0]= 12.*self.c12[0];
    self.cf6[0] = 6.*self.c6[0];
    self.ecut[0]= - (self.c12[0]/self.r2cut**3-self.c6[0])/self.r2cut**3;
    # a-b interaction
    sig6        = (0.5*(siga+sigb))**6
    self.c6[1]  = 4.0*sqrt(epsa*epsb)*sig6;
    self.c12[1] = self.c6[1]*sig6;
    self.cf12[1]= 12.*self.c12[1];
    self.cf6[1] = 6.*self.c6[1];
    self.ecut[1]= - (self.c12[1]/self.r2cut**3-self.c6[1])/self.r2cut**3;
    # b-b interaction
    sig6        = sigb**6
    self.c6[2]  = 4.0*epsb*sig6;
    self.c12[2] = self.c6[2]*sig6;
    self.cf12[2]= 12.*self.c12[2];
    self.cf6[2] = 6.*self.c6[2];
    self.ecut[2]= - (self.c12[2]/self.r2cut**3-self.c6[2])/self.r2cut**3;
    # 

  
  def fcc(self):
    ax = self.Lx/self.l1
    ay = self.Ly/self.l2 
    #ay = self.Ly/(self.l2 + 2*self.dslice) 
    az = self.Lz/self.l3
    print( "# lattice lenghts (ax,ay,az) =", (ax, ay, az) )
    print( "# (l1, l2, l3) =", (self.l1,self.l2, self.l3) )
    mm = self.l1*self.l2*self.l3
    natom = 4*mm  
    print( "# number of lattice cells =", mm )
    print( "# number of particles =" , natom )
    print( "# md-box sides [Lx, Ly, Lz ]=", (self.Lx, self.Ly, self.Lz) )
    j  = 0
    xi = 0.
    yi = 0.
    zi = 0.
    delta=0.005
    
    rrx = random.normal(0., delta, natom)
    rry = random.normal(0., delta, natom)
    rrz = random.normal(0., delta, natom)
    #with open("fcc.txt", "w") as f:
    for nx in range(self.l1) :
        for ny in range(self.l2) :
            for nz in range(self.l3) :
                self.x[j] = xi + ax*nx + rrx[j]
                self.y[j] = yi + ay*ny + rry[j]             
                self.z[j] = zi + az*nz + rrz[j]
                #f.write( "  %d   %8.3f   %8.3f   %8.3f \n" % (j, self.x[j], self.y[j], self.z[j]) )
                #print( (j, self.x[j], self.y[j], self.z[j]) )
                j +=1
                self.x[j] = xi + ax*nx + rrx[j] + 0.5*ax
                self.y[j] = yi + ay*ny + rry[j] + 0.5*ay     
                self.z[j] = zi + az*nz + rrz[j]
                #f.write( "  %d   %8.3f   %8.3f   %8.3f \n" % (j, self.x[j], self.y[j], self.z[j]) )
                #print( (j, self.x[j], self.y[j], self.z[j]) )
                j +=1
                self.x[j] = xi + ax*nx + rrx[j] + 0.5*ax
                self.y[j] = yi + ay*ny + rry[j]             
                self.z[j] = zi + az*nz + rrz[j] + 0.5*az
                #f.write( "  %d   %8.3f   %8.3f   %8.3f \n" % (j, self.x[j], self.y[j], self.z[j]) )
                #print( (j, self.x[j], self.y[j], self.z[j]) )
                j +=1
                self.x[j] = xi + ax*nx + rrx[j] 
                self.y[j] = yi + ay*ny + rry[j] + 0.5*ay            
                self.z[j] = zi + az*nz + rrz[j] + 0.5*az
                #f.write( "  %d   %8.3f   %8.3f    %8.3f \n" % (j, self.x[j], self.y[j], self.z[j]) )
                #print( (j, self.x[j], self.y[j], self.z[j]) )
                j +=1
    print( "# end of initial fcc lattice construction npart =",j)
  

  def calcexy(self, N) :
    # zeroing
    # nb: rectangular PBC
    enex= 0.
    egr = 0.
    viex= 0.
    eps = 1.   # reference Argon 119 K
    sig6= 1.   # reference Argon 0.3405 nm
    #ay  = self.Ly/(self.l2+2*self.dslice)
    ywu = 0.93 + self.l2*self.Ly/(self.l2+2*self.dslice)
    ywd = 0.
    c3  = 4.0*eps*sig6
    c9  = c3*sig6
    c3 *= self.rho*pi/6.
    c9 *= self.rho*pi/45.
    cf3 = 3.*c3
    cf9 = 9.*c9
    ecut= - (c9/self.rcut**6-c3)/self.rcut**3
    # loop over particles inside selected cell
    for i in range(N):
        #dx = (self.rx[i]+0.5)*self.Lx
        dy =  self.ry[i]*self.Ly
        self.fy[i] += self.gforce
        enex += self.gforce*dy
        #dz = (self.rx[i]+0.5)*self.Lz
        dyu = ywu - dy 
        dyd = dy - ywd
        if(dyu < self.rcut) :
            rr1 = 1./dyu
            rr3 = rr1*rr1*rr1
            rr6 = rr3*rr3
            ej  = (c9*rr6 -c3)*rr3 + ecut
            vir = (cf9*rr6 -cf3)*rr3
            enex+= ej
            viex+= vir
            # forces
            self.fy[i] -= vir*rr1
        elif(dyd < self.rcut) :
            rr1 = 1./dyd
            rr3 = rr1*rr1*rr1
            rr6 = rr3*rr3
            ej  = (c9*rr6 -c3)*rr3 + ecut
            vir = (cf9*rr6 -cf3)*rr3
            enex+= ej
            viex+= vir
            # forces
            self.fy[i] += vir*rr1
            ## observable --> stress tensor
            #s1yy[i]+=vir*dyu
            #s1xy[j]+=vir*dx
            #s1yz[i]+=vir*dz
    return ( enex, viex)

  def eqmdi(self, N, Na, mx, my, mz, kt, pas, mode):
    # initial evaluation of forces, energies and virials
    self.rx  =  self.x/self.Lx
    self.rx -= rint(self.rx)
    self.ry  =  self.y/self.Ly
    self.ry -= rint(self.ry)
    self.rz  =  self.z/self.Lz
    self.rz -= rint(self.rz )
    (enep, virial)=calcener(mx, my, mz, N,self.rx,self.ry,self.rz,self.Lx,self.Ly,self.Lz,self.sp,self.r2cut,self.c12,self.c6,self.ecut,self.cf12,self.cf6,self.fx,self.fy,self.fz,self.etxx,self.stxx,self.styy,self.stzz,self.stxy,self.stxz,self.styz)
    print("test 1",enep,virial)
    
    #self.ry e self.fy sono già vettori e possiamo già inserirli nel kernel
    #(enex, virex)  = self.calcexy(N)
    #print("test 2",enex,virex)
    
    if mode == 2 : 
        # andersen thermostats: velocity rescaling
        pstd=sqrt(self.ma*kt)
        self.px[0:Na] = random.normal(0., pstd, Na)
        self.py[0:Na] = random.normal(0., pstd, Na)
        self.pz[0:Na] = random.normal(0., pstd, Na)
        pstd=sqrt(self.mb*kt)
        self.px[Na:N] = random.normal(0., pstd, N-Na)
        self.py[Na:N] = random.normal(0., pstd, N-Na)
        self.pz[Na:N] = random.normal(0., pstd, N-Na)
        vcmx  = sum(self.px)
        vcmy  = sum(self.py)
        vcmz  = sum(self.pz)
        self.px[0:Na]   -= self.ma*vcmx/(Na*self.ma+(N-Na)*self.mb)
        self.py[0:Na]   -= self.ma*vcmy/(Na*self.ma+(N-Na)*self.mb)
        self.pz[0:Na]   -= self.ma*vcmz/(Na*self.ma+(N-Na)*self.mb)
        self.px[Na:N]   -= self.mb*vcmx/(Na*self.ma+(N-Na)*self.mb)
        self.py[Na:N]   -= self.mb*vcmy/(Na*self.ma+(N-Na)*self.mb)
        self.pz[Na:N]   -= self.mb*vcmz/(Na*self.ma+(N-Na)*self.mb)
        print("# velocities sampled from maxwell distribution at timestep" , pas)
    vcmx = sum(self.px)
    vcmy = sum(self.py)
    vcmz = sum(self.pz)
    enek = 0.5*sum( (self.px**2+self.py**2+self.pz**2)/self.m )
    self.ept  = 0.
    self.ekt  = 0.
    self.pres = 0.
    return (enep/N, enek/N, virial/N, vcmx, vcmy, vcmz)   


  def eqmdr(self, N, Na, mx, my, mz, kt, pas, dt, freq):
    dth=0.5*dt
    for ip in range(freq):
        pas+= 1
        t   = pas*dt
        # momenta first 
        self.px[0:N] += self.fx[0:N]*dth
        self.py[0:N] += self.fy[0:N]*dth
        self.pz[0:N] += self.fz[0:N]*dth        
        # positions second
        self.x[0:N]  += dt*self.px[0:N]/self.m[0:N]
        self.y[0:N]  += dt*self.py[0:N]/self.m[0:N]
        self.z[0:N]  += dt*self.pz[0:N]/self.m[0:N]
        # compute forces
        self.rx[0:N] =  self.x[0:N]/self.Lx
        self.rx[0:N]-= rint(self.rx[0:N])
        self.ry[0:N] =  self.y[0:N]/self.Ly
        self.ry[0:N]-= rint(self.ry[0:N])
        self.rz[0:N] =  self.z[0:N]/self.Lz
        self.rz[0:N]-= rint(self.rz[0:N])
        (enep, virial)=calcener(mx,my,mz,N,self.rx,self.ry,self.rz,self.Lx,self.Ly,self.Lz,self.sp,self.r2cut,self.c12,self.c6,self.ecut,self.cf12,self.cf6,self.fx,self.fy,self.fz,self.etxx,self.stxx,self.styy,self.stzz,self.stxy,self.stxz,self.styz)

        #self.ry e self.fy sono già vettori e possiamo già inserirli nel kernel
        #(enex, virex)  = self.calcexy(N)
        #print("test 3",enex,virex)

        # momenta third
        self.px[0:N] += self.fx[0:N]*dth
        self.py[0:N] += self.fy[0:N]*dth
        self.pz[0:N] += self.fz[0:N]*dth   
        # one step advanced 
        vcmx = sum(self.px)
        vcmy = sum(self.py)
        vcmz = sum(self.pz)
        enek = 0.5*sum( (self.px**2+self.py**2+self.pz**2)/self.m )
        self.ekt += enek
        self.ept += enep
        self.pres+= virial       
    return (t, enep/N, enek/N, virial/N, vcmx, vcmy, vcmz )     


  def nemdi(self, N, Na, mx, my, mz, kt, dkt, pas, lther):
    tlef = 3.*(kt + dkt)
    trig = 3.*kt
    # initial evaluation of forces, energies and virials
    self.rx  =  self.x/self.Lx
    self.rx -= rint(self.rx)
    self.ry  =  self.y/self.Ly
    self.ry -= rint(self.ry)
    self.rz  =  self.z/self.Lz
    self.rz -= rint(self.rz )
    (enep, virial)=calcener(mx,my,mz,N,self.rx,self.ry,self.rz,self.Lx,self.Ly,self.Lz,self.sp,self.r2cut,self.c12,self.c6,self.ecut,self.cf12,self.cf6,self.fx,self.fy,self.fz,self.etxx,self.stxx,self.styy,self.stzz,self.stxy,self.stxz,self.styz)
    #self.ry e self.fy sono già vettori e possiamo già inserirli nel kernel
    #(enex, virex)  = self.calcexy(N)

    # initializing soret thermostats: velocity rescaling
    hista,histb,eneka,enekb,heat,gz,jmaz,jez,ene = self.therm(tlef,trig,N,Na,lther)
   #
    self.heatt[:,pas] =  (heat[:])
   #
    self.jmazt[:,pas] =  ( jmaz[:])
    self.gzt[:,pas]   =  (   gz[:])
    self.jezt[:,pas]  =  (  jez[:])
   #
    self.hstat[:,pas] =  (hista[:])
    self.hstbt[:,pas] =  (histb[:])
    self.enkat[:,pas] =  (eneka[:])
    self.enkbt[:,pas] =  (enekb[:])
    self.enet[:,pas]  =  (  ene[:])
#
    vcmx = sum(self.px)
    vcmy = sum(self.py)
    vcmz = sum(self.pz)
    enek = 0.5*sum( (self.px**2+self.py**2+self.pz**2)/self.m )
    # initializing counters and constants
    self.ept  = 0.
    self.ekt  = 0.
    self.pres = 0.
    ## print("# starting dnemd trajectory")
    ## print( "( 'pas', 'enep', 'enek', 'enet', 'heatin', 'heatout', 'vcm' )")
    ## print( (0., enep/N, enek/N, (enep+enek)/N, heat[0], heat[1], vcmx, vcmy, vcmz) )
    return (enep/N, enek/N, (enep+enek)/N, heat[0], heat[1], vcmx, vcmy, vcmz)
    
  def nemdr(self, N, Na, mx, my, mz, kt, dkt, pas, dt, freq, lther):
    tlef = 3.*(kt + dkt)
    trig = 3.*kt
    dth=0.5*dt
    for ip in range(freq):
        pas+= 1
        t   = pas*dt
        # advance one step
        # momenta first 
        self.px[0:N] += self.fx[0:N]*dth
        self.py[0:N] += self.fy[0:N]*dth
        self.pz[0:N] += self.fz[0:N]*dth        
        # positions second
        self.x[0:N]  += dt*self.px[0:N]/self.m[0:N]
        self.y[0:N]  += dt*self.py[0:N]/self.m[0:N]
        self.z[0:N]  += dt*self.pz[0:N]/self.m[0:N]
        # compute forces
        self.rx[0:N] = self.x[0:N]/self.Lx
        self.rx[0:N]-= rint(self.rx[0:N])
        self.ry[0:N] = self.y[0:N]/self.Ly
        self.ry[0:N]-= rint(self.ry[0:N])
        self.rz[0:N] = self.z[0:N]/self.Lz
        self.rz[0:N]-= rint(self.rz[0:N])
        (enep, virial)=calcener(mx,my,mz,N,self.rx,self.ry,self.rz,self.Lx,self.Ly,self.Lz,self.sp,self.r2cut,self.c12,self.c6,self.ecut,self.cf12,self.cf6,self.fx,self.fy,self.fz,self.etxx,self.stxx,self.styy,self.stzz,self.stxy,self.stxz,self.styz)
        #self.ry e self.fy sono già vettori e possiamo già inserirli nel kernel
        #(enex, virex)  = self.calcexy(N)

        # momenta third
        self.px[0:N] += self.fx[0:N]*dth
        self.py[0:N] += self.fy[0:N]*dth
        self.pz[0:N] += self.fz[0:N]*dth   
        # one step advanced 
        # soret thermostats: velocity rescaling
        hista,histb,eneka,enekb,heat,gz,jmaz,jez,ene = self.therm(tlef,trig,N,Na,lther)
       #
        self.heatt[:,pas] = (heat[:])
       #
        self.jmazt[:,pas] = ( jmaz[:])
        self.gzt[:,pas]   = (   gz[:])
        self.jezt[:,pas]  = (  jez[:])
       #
        self.hstat[:,pas] = (hista[:])
        self.hstbt[:,pas] = (histb[:])
        self.enkat[:,pas] = (eneka[:])
        self.enkbt[:,pas] = (enekb[:])
        self.enet[:,pas]  = (  ene[:])
       #
        #end inner --> single step printout
        vcmx = sum(self.px)
        vcmy = sum(self.py)
        vcmz = sum(self.pz)
        enek = 0.5*sum( (self.px**2+self.py**2+self.pz**2)/self.m )
        self.ekt += enek
        self.ept += enep
        self.pres+= virial        
        ## if (pas)%freq==0 : 
            ## fout.write (" %8.3f %9.4f %9.4f %10.7f %8.3f %8.3f  %7.2g %7.2g %7.2g \n" % \
    ## print( (t, enep/N, enek/N, (enep+enek)/N, heat[0], heat[1], vcmx, vcmy, vcmz) )
    return (t, enep/N, enek/N, (enep+enek)/N, heat[0], heat[1], vcmx, vcmy, vcmz)
   # end of ne-md run
    

  def therm(self,tlef,trig,N,Na,lther) :
    # thermostats: velocity rescaling
    nindl= zeros(N, dtype=int32 )
    nla  = zeros(self.lb)
    nlb  = zeros(self.lb)
    eca  = zeros(self.lb)
    ecb  = zeros(self.lb)
    vx   = zeros(self.lb)
    vy   = zeros(self.lb)
    vz   = zeros(self.lb)
    jmaz = zeros(self.lb)
    jmz  = zeros(self.lb)
    jez  = zeros(self.lb)
    e    = zeros(self.lb)
    ppx  = zeros(N)
    ppy  = zeros(N)
    ppz  = zeros(N)
    mvx  = zeros(N)
    mvy  = zeros(N)
    mvz  = zeros(N)
    heat = zeros(2)
    for i in range(Na) :
        li = int((self.rz[i]+0.5)*self.lb) 
        nla[li] += 1.
        nindl[i] = li
        vx[li]+= self.px[i]
        vy[li]+= self.py[i]
        vz[li]+= self.pz[i]
    for i in range(Na,N) :
        li = int((self.rz[i]+0.5)*self.lb) 
        nlb[li] += 1.
        nindl[i] = li
        vx[li]+= self.px[i]
        vy[li]+= self.py[i]
        vz[li]+= self.pz[i]
    ml = zeros(self.lb)
    for li in range(self.lb) :
        ml[li] = (nla[li]*self.m[0]+nlb[li]*self.m[Na])
    for i in range(Na) :
        li = nindl[i] 
        mvx[i]  = self.m[i] *vx[li]/ml[li]
        ppx[i]  = self.px[i]-mvx[i]
        mvy[i]  = self.m[i] *vy[li]/ml[li]
        ppy[i]  = self.py[i]-mvy[i]
        mvz[i]  = self.m[i] *vz[li]/ml[li]
        ppz[i]  = self.pz[i]-mvz[i]
        eca[li]+=(ppx[i]**2 + ppy[i]**2 + ppz[i]**2) 
    eca /=self.m[0]
    for i in range(Na,N) :
        li = nindl[i] 
        mvx[i]  = self.m[i] *vx[li]/ml[li]
        ppx[i]  = self.px[i]-mvx[i]
        mvy[i]  = self.m[i] *vy[li]/ml[li]
        ppy[i]  = self.py[i]-mvy[i]
        mvz[i]  = self.m[i] *vz[li]/ml[li]
        ppz[i]  = self.pz[i]-mvz[i]
        ecb[li]+=(ppx[i]**2 + ppy[i]**2 + ppz[i]**2) 
    ecb /=self.m[Na]
    # thermostatting 'lther' layers on the left side of the MD-box
    eclef= zeros(self.lb)
    ecrig= zeros(self.lb)
    nlef = zeros(self.lb)
    nrig = zeros(self.lb)
    lfact= zeros(self.lb)
    rfact= zeros(self.lb)
    eclef[0:lther]= eca[0:lther] + ecb[0:lther]
    nlef[0:lther] = nla[0:lther] + nlb[0:lther]
    lfact[0:lther]=     tlef*(nlef[0:lther]-1.)/eclef[0:lther]
    heat[0]       = sum(    (lfact[0:lther]-1.)*eclef[0:lther])
    eca[0:lther] *= lfact[0:lther]
    ecb[0:lther] *= lfact[0:lther]
    lfact[0:lther]=sqrt(lfact[0:lther])
#
    for i in range(N) :
        li = nindl[i]
        if  li < lther :
            ppx[i]*= lfact[li]
            ppy[i]*= lfact[li]
            ppz[i]*= lfact[li]
            self.px[i]  = ppx[i]+mvx[i]
            self.py[i]  = ppy[i]+mvy[i]
            self.pz[i]  = ppz[i]+mvz[i]
# put back all analysis
    for i in range(N) :
        self.etxx[i] += (ppx[i]**2 + ppy[i]**2 + ppz[i]**2)/self.m[i]
    for i in range(Na) :
        li = nindl[i] 
        e[li]    += self.etxx[i]
        jmaz[li] += ppz[i]
        jez[li] += 0.5*(ppz[i]*self.etxx[i] + self.stxz[i]*ppx[i] + self.styz[i]*ppy[i] + self.stzz[i]*ppz[i])/self.m[i]
    for i in range(Na,N) :
        li = nindl[i] 
        e[li]    += self.etxx[i]
        jez[li] += 0.5*(ppz[i]*self.etxx[i] + self.stxz[i]*ppx[i] + self.styz[i]*ppy[i] + self.stzz[i]*ppz[i])/self.m[i]
    #    
    return (nla, nlb ,eca, ecb, heat, vz, jmaz, jez, e)

  def read_input(self, N, conf_in='conf_in.b'): #, mom_in='mom_in'):
      with open(conf_in, 'rb') as ftrj:
          (Nr, pas) = load(ftrj)
          if N!=Nr :
              #print(' reading %d particle configuration from step %d' % (Nr,pas) )
          #else :
              print(' ??? reading %d particle configuration expected %d' % (Nr,N) )
          ( self.x,  self.y,  self.z ) = load( ftrj)
          ( self.px, self.py, self.pz) = load( ftrj)
      return pas

  def write_input(self, N, pas, conf_out='conf_in.b'): #, mom_out='mom_in'):
      with open(conf_out, 'wb') as ftrj:
          dump( (N, pas) , ftrj, HIGHEST_PROTOCOL)
          dump( ( self.x,  self.y,  self.z ), ftrj, HIGHEST_PROTOCOL)
          dump( ( self.px, self.py, self.pz), ftrj, HIGHEST_PROTOCOL)
               
  def dumpxyz(self, N, Na, pas, dumpf):
      dx = self.x/self.Lx
      dx-= rint(dx)
      dy = self.y/self.Ly
      dy-= rint(dy)
      self.y = dy*self.Ly
      dz = self.z/self.Lz
      dz-= rint(dz)
      ar = empty(N,(str_,2))
      sig=3.4 # in Angstroem for argon   
      ar[0:Na] = "Ar"
      ar[Na:N] = "Ar"
      dx *= sig*self.Lx
      dy *= sig*self.Ly
      dz *= sig*self.Lz
      dumpf.write( " %d \n" % N ) 
      dumpf.write( " %d \n" % pas ) 
      for i in range(N):
          dumpf.write( "%s  %10.5f   %10.5f  %10.5f\n" % (ar[i], dx[i], dy[i], dz[i]) )
            
  def writexyz(self, N, Na):
      dx = self.x/self.Lx
      dx-= rint(dx)
      self.x = dx*self.Lx
      dy = self.y/self.Ly
      dy-= rint(dy)
      self.y = dy*self.Ly
      dz = self.z/self.Lz
      dz-= rint(dz)
      self.z = dz*self.Lz
      ar = empty(N,(str_,2))
      sig=3.4 # in Angstroem for argon   
      ar[0:Na] = "Ar"
      ar[Na:N] = "Ar"
      dx *= sig*self.Lx
      dy *= sig*self.Ly
      dz *= sig*self.Lz
      rout = column_stack( (ar, dx, dy, dz) )
      savetxt('conf.xyz', rout, fmt=(' %s', '%s ','%s','%s'), \
      header=(' %d \n' % N ), comments=' ' )          

  def write_out(self, N, Na, tstep, gdr_out='gdr.out'):
      V = zeros(self.kg) 
      r = zeros(self.kg)
      g = zeros( (self.kg,3) ) 
      for lm in range(self.kg) :
          V[lm] = 4./3.*pi*(self.ldel**3)*(3*lm*lm +3*lm + 1); 
          r[lm] = (lm+0.5)*self.ldel
      g[:,0] = self.gcount[:,0]/(V*(Na-1)*tstep*self.rho*Na/N);
      g[:,1] = self.gcount[:,1]/(2.*V*Na*(N-Na)*tstep*self.rho/N);
      g[:,2] = self.gcount[:,2]/(V*(N-(Na-1))*tstep*self.rho*(N-Na)/N);
      gout = column_stack( (r, g) )
      savetxt(gdr_out, gout , fmt=('%10.5g ','%12.7g','%12.7g','%12.7g'), \
      header="    'r'     'gaa'     'gab'     'gbb' " )        


### nvtg_write

In [7]:
from numpy import zeros, pi, sqrt, rint, int64, float64,int32
from numpy import savetxt, column_stack, empty, str_
from numpy import pi
from numpy import random
from pickle import dump, load, HIGHEST_PROTOCOL

def read_input(self, N, conf_in='conf_in.b'): #, mom_in='mom_in'):
    with open(conf_in, 'rb') as ftrj:
        (Nr, pas) = load(ftrj)
        if N!=Nr :
            #print(' reading %d particle configuration from step %d' % (Nr,pas) )
        #else :
            print(' ??? reading %d particle configuration expected %d' % (Nr,N) )
        ( self.x,  self.y,  self.z ) = load( ftrj)
        ( self.px, self.py, self.pz) = load( ftrj)
    return pas

def write_input(self, N, pas, conf_out='conf_in.b'): #, mom_out='mom_in'):
    with open(conf_out, 'wb') as ftrj:
        dump( (N, pas) , ftrj, HIGHEST_PROTOCOL)
        dump( ( self.x,  self.y,  self.z ), ftrj, HIGHEST_PROTOCOL)
        dump( ( self.px, self.py, self.pz), ftrj, HIGHEST_PROTOCOL)
               
def dumpxyz(self, N, Na, pas, dumpf):
    dx = self.x/self.Lx
    dx-= rint(dx)
    dy = self.y/self.Ly
    #dy-= rint(dy)
    self.y = dy*self.Ly
    dz = self.z/self.Lz
    dz-= rint(dz)
    ar = empty(N,(str_,2))
    sig=3.4 # in Angstroem for argon   
    ar[0:Na] = "Ar"
    ar[Na:N] = "Ar"
    dx *= sig*self.Lx
    dy *= sig*self.Ly
    dz *= sig*self.Lz
    dumpf.write( " %d \n" % N ) 
    dumpf.write( " %d \n" % pas ) 
    for i in range(N):
        dumpf.write( "%s  %10.5f   %10.5f  %10.5f\n" % (ar[i], dx[i], dy[i], dz[i]) )
            
def writexyz(self, N, Na):
    dx = self.x/self.Lx
    dx-= rint(dx)
    self.x = dx*self.Lx
    dy = self.y/self.Ly
    dy-= rint(dy)
    self.y = dy*self.Ly
    dz = self.z/self.Lz
    dz-= rint(dz)
    self.z = dz*self.Lz
    ar = empty(N,(str_,2))
    sig=3.4 # in Angstroem for argon   
    ar[0:Na] = "Ar"
    ar[Na:N] = "Ar"
    dx *= sig*self.Lx
    dy *= sig*self.Ly
    dz *= sig*self.Lz
    rout = column_stack( (ar, dx, dy, dz) )
    savetxt('conf.xyz', rout, fmt=(' %s', '%s ','%s','%s'), \
    header=(' %d \n' % N ), comments=' ' )          

def write_out(self, N, Na, tstep, gdr_out='gdr.out'):
    V = zeros(self.kg) 
    r = zeros(self.kg)
    g = zeros( (self.kg,3) ) 
    for lm in range(self.kg) :
        V[lm] = 4./3.*pi*(self.ldel**3)*(3*lm*lm +3*lm + 1); 
        r[lm] = (lm+0.5)*self.ldel
    g[:,0] = self.gcount[:,0]/(V*(Na-1)*tstep*self.rho*Na/N);
    g[:,1] = self.gcount[:,1]/(2.*V*Na*(N-Na)*tstep*self.rho/N);
    g[:,2] = self.gcount[:,2]/(V*(N-(Na-1))*tstep*self.rho*(N-Na)/N);
    gout = column_stack( (r, g) )
    savetxt(gdr_out, gout , fmt=('%10.5g ','%12.7g','%12.7g','%12.7g'), \
    header="    'r'     'gaa'     'gab'     'gbb' " )          



### System setup: object instantiation 

In [13]:
from numpy import zeros, pi, sin, cos, empty, array, float64
#from nvtg_lj import LJ
#from nvtg_write import *
from time import process_time
from  pickle import dump, load, HIGHEST_PROTOCOL

# from numba import jit
if __name__ == "__main__":
    dir="./RUN/"
# r-parameters # defaults
    rho  = 0.984   #  density of the solid phase
    mx   = 3#6       #  number of cells along x
    my   = 5#6 + 2   #  number of cells along y
    mz   = 7#12+ 2   #  number of cells along z
    kt   = 0.7     #  temperature
    dk0  = 0.00    #  delta T at equilibrium
    dkt  = 0.70    #  delta T for melting region
    dt   = 0.005   #  timestep (LJ units)
    gforce = -0.1  #  vaule of gravity field
    freq =  2      #  printing frequency
    lther=  3      #  width of thermostat region
    nstep= 10      #  number of multiples of 'freq' time steps 
    pfile = dir + "fccmd.pickle"
    print( "# number of boxes mx = %d, my = %d, mz = %d" % (mx, my, mz) )
    print( "# mean (kinetic) temperature kt = %8.4f " % (kt) )
    print( "# integration time step dt = %8.4f" % dt )
    print( "# number of time steps for this run nstep = %d" % (nstep) )    
    t0 = process_time()
    IN = LJ(rho, mx, my, mz, gforce, nstep*freq)
    print("# creating EQ object, cpu = %8.4f" % (process_time()-t0) )
    N = IN.npart
    Na = 3*N // 25
    rhofact = IN.lb / (IN.Lx * IN.Ly * IN.Lz)
    print( "# sides of the rectangular MD-box L = [ %8.4f %8.4f %8.4f ]" % (IN.Lx, IN.Ly, IN.Lz) )
    print( "# number of particles  N = %d" % N)
    print( "# density rho = %8.4f   (%8.4f)" % (IN.rho, rho))
    print( "# potential cut-off radius  rcut = %8.4f *sigma" % IN.rcut)
    print( "# lateral size of slicing Deltaz = %8.4f" % (IN.Lz / IN.lb))

# number of boxes mx = 3, my = 5, mz = 7
# mean (kinetic) temperature kt =   0.7000 
# integration time step dt =   0.0050
# number of time steps for this run nstep = 10
# creating EQ object, cpu =   0.0029
# sides of the rectangular MD-box L = [   9.5758  15.9596  22.3434 ]
# number of particles  N = 3360
# density rho =   1.6400   (  0.9840)
# potential cut-off radius  rcut =   3.0000 *sigma
# lateral size of slicing Deltaz =   1.5960


In [14]:
    # starting from (fcc) lattice
    IN.fcc()
    pas = 0
    #write_input(IN, N, pas, conf_out=dir+"conf_fcc0984.b")
    writexyz(IN, N, Na)

# lattice lenghts (ax,ay,az) = (1.5959586353904498, 1.5959586353904496, 1.5959586353904496)
# (l1, l2, l3) = (6, 10, 14)
# number of lattice cells = 840
# number of particles = 3360
# md-box sides [Lx, Ly, Lz ]= (9.575751812342698, 15.959586353904495, 22.343420895466295)
# end of initial fcc lattice construction npart = 3360


In [15]:
    mode=2
    #pas = read_input(IN, N, conf_in=dir+"conf_fcc0984.b")
    t0 = process_time()    
    (enep, enek, virial, vcmx, vcmy, vcmz) = IN.eqmdi( N, Na, mx, my, mz, kt, pas, mode)
    print( "# initial conditions step = %d, cpu = %8.4f" % (pas, process_time()-t0) )

test 1 -25816.790574206243 -47307.619291771785
# velocities sampled from maxwell distribution at timestep 0
# initial conditions step = 0, cpu =   0.8231


In [16]:
    write_input(IN, N, nstep, conf_out=dir+"conf_fcc0984.b")

In [17]:
    %%time
    nstep= 10      #  number of multiples of 'freq' time steps 
    # Equilibrium run with temperature from maxwellian sampling
    print("# starting eqmd trajectory\n")
    print( "    'pas'   'enep'     'enek'   'enet'      'virial'   'vcm' ")
    print(" %9.2f %9.4f %9.4f %12.7f %8.3f %12.2g %7.2g %7.2g" % (pas*dt,enep, enek, enep+enek, virial, vcmx, vcmy, vcmz) )
    t0 = process_time()
    for it in range(nstep):
        (t, enep, enek, virial, vcmx, vcmy, vcmz) = IN.eqmdr( N, Na, mx, my, mz, kt, pas, dt, freq)
        print(" %9.2f %9.4f %9.4f %12.7f %8.3f %12.2g %7.2g %7.2g" % (t,enep, enek, enep+enek, virial, vcmx, vcmy, vcmz) )
        pas += freq    
    print( "# (IN)-Equilibrium run, cpu = %8.4f" % (process_time()-t0) )
    g=3*N-3
    nstep *= freq
    print( "# ending initial equilibrium run  <ep>=%6.1f  <ek>=%7.4f   T=%8.3f  P=%8.3f\n" % ( IN.ept/nstep, IN.ekt/nstep, 2.*IN.ekt/(nstep*g), IN.pres/nstep) )
    # final positions and momenta to file conf_in.b
    write_input(IN, N, nstep, conf_out=dir+"conf_eg0984.b")
    writexyz(IN, N, Na)

# starting eqmd trajectory

    'pas'   'enep'     'enek'   'enet'      'virial'   'vcm' 
      0.00   -7.6836    1.0571   -6.6264562  -14.080     -2.7e-14 2.8e-14  -5e-14
      0.01   -7.6626    1.0362   -6.6264591  -13.599     -1.8e-14 2.8e-14 -4.3e-14
      0.02   -7.6007    0.9743   -6.6264775  -12.390     -3.2e-14 3.6e-14  -5e-14
      0.03   -7.4993    0.8728   -6.6264225  -10.560     -4.3e-14   5e-14 -5.7e-14
      0.04   -7.3650    0.7388   -6.6262065   -8.228     -4.1e-14 5.3e-14 -4.6e-14
      0.05   -7.2150    0.5892   -6.6257941   -5.683     -4.4e-14   5e-14  -5e-14
      0.06   -7.0768    0.4514   -6.6253395   -3.366     -5.4e-14   5e-14 -4.6e-14
      0.07   -6.9789    0.3538   -6.6251184   -1.719     -9.2e-14 4.3e-14 -6.4e-14
      0.08   -6.9354    0.3101   -6.6252267   -0.955     -9.2e-14 1.4e-14 -8.2e-14
      0.09   -6.9400    0.3145   -6.6255212   -0.985     -9.9e-14 1.8e-14 -8.5e-14
      0.10   -6.9744    0.3486   -6.6258166   -1.523       -8e-14 3.6e-15 -7.8e-14


In [ ]:
    freq = 20
    nstep= 10       #  number of multiples of 'freq' time steps 
    mode = 2
    print("check gforce=",IN.gforce)
    t0 = process_time()    
    pas = read_input(IN, N, conf_in=dir+"conf_eg0984.b")
    (enep, enek, virial, vcmx, vcmy, vcmz) = IN.eqmdi( N, Na, mx, my, mz, kt, pas, mode)
    print( "# initial conditions step = %d, cpu = %8.4f" % (pas, process_time()-t0) )    
    # Equilibrium run with temperature from maxwellian sampling
    print("# starting eqmd trajectory\n")
    print( "    'pas'   'enep'     'enek'   'enet'      'virial'   'vcm' ")
    print("%8.2f  %9.4f %9.4f %12.7f %8.3f %12.2g %7.2g %7.2g" % (pas*dt,enep, enek, enep+enek, virial, vcmx, vcmy, vcmz) )
    t0 = process_time()
    for it in range(nstep):
        (t, enep, enek, virial, vcmx, vcmy, vcmz) = IN.eqmdr( N, Na, mx, my, mz, kt, pas, dt, freq)
        print("%8.2f  %9.4f %9.4f %12.7f %8.3f %12.2g %7.2g %7.2g" % (t,enep, enek, enep+enek, virial, vcmx, vcmy, vcmz) )
        pas += freq    
    print( "# (IN)-Equilibrium run, cpu = %8.4f" % (process_time()-t0) )
    g=3*N-3
    nstep *= freq
    print( "# ending initial equilibrium run  <ep>=%6.1f  <ek>=%7.4f   T=%8.3f  P=%8.3f\n" % ( IN.ept/nstep, IN.ekt/nstep, 2.*IN.ekt/(nstep*g), IN.pres/nstep) )
    # final positions and momenta to file conf_in.b
    write_input(IN, N, nstep, conf_out=dir+"conf_eg0984.b")
    writexyz(IN, N, Na)

check gforce= -0.1
test 1 -23461.31150861997 -5701.6462352911085
# velocities sampled from maxwell distribution at timestep 20
# initial conditions step = 20, cpu =   0.7830
# starting eqmd trajectory

    'pas'   'enep'     'enek'   'enet'      'virial'   'vcm' 
    0.10    -6.9825    1.0509   -5.9316769   -1.697     -3.6e-15  -6e-14 -4.8e-14
    0.20    -6.8794    0.9479   -5.9314855   -0.060      1.2e-14 -8.5e-14 -1.1e-13


Exception ignored in: <finalize object at 0x7fd7eafc9a30; dead>
Traceback (most recent call last):
  File "/usr/lib/python3.7/weakref.py", line 572, in __call__
    return info.func(*info.args, **(info.kwargs or {}))
  File "/usr/local/lib/python3.7/dist-packages/numba/cuda/cudadrv/driver.py", line 1741, in deref
    mem.free()
  File "/usr/local/lib/python3.7/dist-packages/numba/cuda/cudadrv/driver.py", line 1589, in free
    self._finalizer()
  File "/usr/lib/python3.7/weakref.py", line 572, in __call__
    return info.func(*info.args, **(info.kwargs or {}))
  File "/usr/local/lib/python3.7/dist-packages/numba/cuda/cudadrv/driver.py", line 1257, in core
    deallocations.add_item(driver.cuMemFree, handle, size)
  File "/usr/local/lib/python3.7/dist-packages/numba/cuda/cudadrv/driver.py", line 958, in add_item
    self.clear()
  File "/usr/local/lib/python3.7/dist-packages/numba/cuda/cudadrv/driver.py", line 969, in clear
    dtor(handle)
  File "/usr/local/lib/python3.7/dist-packages

    0.30    -6.8430    0.9116   -5.9313998    0.546      4.9e-14 -7.3e-14 -9.9e-14
    0.40    -6.8474    0.9160   -5.9313442    0.495      1.3e-13 -3.2e-14 -1.1e-13
    0.50    -6.8694    0.9380   -5.9314309    0.106      1.8e-13 -8.2e-14 -1.6e-13
    0.60    -6.8674    0.9360   -5.9314521    0.126      2.2e-13 -2.1e-14 -3.2e-13
    0.70    -6.8484    0.9170   -5.9313704    0.437      2.5e-13 -1.6e-14 -3.7e-13
    0.80    -6.8735    0.9422   -5.9313631    0.028      2.8e-13 1.5e-14 -3.7e-13
    0.90    -6.8436    0.9123   -5.9313534    0.497        3e-13 6.8e-14 -4.2e-13
    1.00    -6.8540    0.9227   -5.9313636    0.348      2.6e-13 8.1e-14 -4.6e-13
    1.10    -6.8697    0.9384   -5.9313614    0.082      1.7e-13 3.4e-14 -5.1e-13
# (IN)-Equilibrium run, cpu = 160.3138
# ending initial equilibrium run  <ep>=-23043.2  <ek>=3113.7868   T=   0.618  P= 950.403



In [ ]:
    freq = 20
    nstep= 50      #  number of multiples of 'freq' time steps 
    mode = 1
    # Xe 1.115
    pfile = dir + "eqmd.pickle"
    print( "# number of boxes mx = %d, my = %d, mz = %d" % (mx, my, mz) )
    print( "# mean (kinetic) temperature kt = %8.4f  delta kt= %8.4f" % (kt, dkt) )
    print( "# integration time step dt = %8.4f" % dt )
    print( "# number of time steps for this run nst = %d " % (nstep) )    
    if mode == 1:
        t0 = process_time()
        EQ = LJ(rho, mx, my, mz, gforce, nstep*freq)
        print("# creating NE object, cpu = %8.4f" % (process_time()-t0) )
        N = EQ.npart
        Na = 3*N // 25
        print( "# sides of the rectangular MD-box L = [ %8.4f %8.4f %8.4f ]" % (EQ.Lx, EQ.Ly, EQ.Lz) )
        print( "# number of particles  N = %d" % N)
        print( "# density rho = %8.4f   (%8.4f)" % (EQ.rho, rho) )
        print( "# potential cut-off radius  rcut = %8.4f *sigma" % EQ.rcut)
        print( "# lateral size of slicing Deltaz = %8.4f" % (EQ.Lz / EQ.lb))
        # initial equilibrated condition
        t0 = process_time()
        estep = read_input(EQ, N, conf_in=dir+"conf_eg0984.b")
        print("# initial conditions, cpu = %8.4f" % (process_time()-t0) )
        # Non-Equilibrium run with temperature gradient
        t0 = process_time()
        pas= 0
        (enep, enek, etot, heat0, heat1, vcmx, vcmy, vcmz) = EQ.nemdi( N, Na, mx, my, mz, kt, dk0, pas, lther)
        print("# starting dnemd trajectory\n")
        print( "    'pas'   'enep'     'enek'   'enet'      'heatin - heatout'   'vcm' ")
        print("%8.2f  %9.4f %9.4f %12.7f %8.3f %8.3f %12.2g %7.2g %7.2g" % (0.,enep, enek, etot, heat0, heat1, vcmx, vcmy, vcmz) )
        for it in range(nstep):
            (t, enep, enek, etot, heat0, heat1, vcmx, vcmy, vcmz) = EQ.nemdr( N, Na, mx, my, mz, kt, dk0, pas, dt, freq, lther)
            print("%8.2f  %9.4f %9.4f %12.7f %8.3f %8.3f %12.2g %7.2g %7.2g" % (t,enep, enek, etot, heat0, heat1, vcmx, vcmy, vcmz) )
            pas += freq    
        print( "# (NE)-Equilibrium run, cpu = %8.4f" % (process_time()-t0) )
        g=3*N-3
        nstep *= freq
        print( "# ending ne-md equilibrium trajectory  <ep>=%6.1f  <ek>=%7.4f   T=%8.3f  P=%8.3f\n" % ( EQ.ept/nstep, EQ.ekt/nstep, 2.*EQ.ekt/(nstep*g), EQ.pres/nstep) )
    # final positions and momenta to file conf_in.b
        write_input(EQ, N, nstep, conf_out=dir+"conf_eg0984.b")
        # saving with pickle
        with open(pfile, 'wb') as ftrj:
            rhofact = EQ.lb / (EQ.Lx * EQ.Ly * EQ.Lz)
            dump( (dkt,rhofact,id) , ftrj, HIGHEST_PROTOCOL)
            dump( (EQ.heatt,EQ.hstat,EQ.hstbt,EQ.enet,EQ.enkat,EQ.enkbt,EQ.jmazt,EQ.gzt,EQ.jezt), ftrj, HIGHEST_PROTOCOL)

# number of boxes mx = 3, my = 5, mz = 7
# mean (kinetic) temperature kt =   0.7000  delta kt=   0.7000
# integration time step dt =   0.0050
# number of time steps for this run nst = 50 
# creating NE object, cpu =   0.0030
# sides of the rectangular MD-box L = [   9.5758  15.9596  22.3434 ]
# number of particles  N = 3360
# density rho =   1.6400   (  0.9840)
# potential cut-off radius  rcut =   3.0000 *sigma
# lateral size of slicing Deltaz =   1.5960
# initial conditions, cpu =   0.0034
# starting dnemd trajectory

    'pas'   'enep'     'enek'   'enet'      'heatin - heatout'   'vcm' 
    0.00    -6.8697    0.9498   -5.9199154   76.917    0.000      1.8e-13   2e-14  -5e-13
    0.10    -6.8328    0.9491   -5.8837244   -1.164    0.000      1.8e-13 3.2e-14 -4.9e-13
    0.20    -6.8236    0.9440   -5.8795754    4.393    0.000      1.7e-13 -3.2e-14 -4.8e-13
    0.30    -6.8249    0.9545   -5.8703599   17.197    0.000      1.8e-13 -1.3e-13 -4.3e-13


In [ ]:
    writexyz(IN, N, Na)

## Non-equilibrium dynamics

In [ ]:
from numpy import zeros, pi, sin, cos, empty, array, float64
#from nvtg2_lj import LJ
#from nvtg_write import *
from time import process_time
from  pickle import dump, load, HIGHEST_PROTOCOL

# from numba import jit
if __name__ == "__main__":
    dir="./RUN/"
    pfile = dir + "ngmdA.pickle"
# r-parameters     # defaults
    nstep= 200     #  number of multiples of 'freq' time steps 
    rho  = 0.984   #  density of the solid phase
    mx   = 6       #  number of cells along x
    my   = 6 + 2   #  number of cells along y
    mz   = 12+ 2   #  number of cells along z
    kt   = 0.7     #  temperature
    dk0  = 0.00    #  delta T at equilibrium
    dkt  = 0.70    #  delta T for melting region
    dt   = 0.005   #  timestep (LJ units)
    gforce = -0.1  #  vaule of gravity field
    freq = 10      #  printing frequency
    lther=  3      #  width of thermostat region
    mode = 1
    print( "# number of boxes mx = %d, my = %d, mz = %d" % (mx, my, mz) )
    print( "# base (kinetic) temperature kt = %8.4f  delta kt= %8.4f" % (kt, dkt) )
    print( "# integration time step dt = %8.4f" % dt )
    print( "# number of time steps for this run nst = %d " % (nstep) )    
    with open(dir+"trajectoryAg.xyz", 'w') as dumpf:
        t0 = process_time()
        NE = LJ(rho, mx, my, mz, gforce, nstep*freq)
        print("# creating NE object, cpu = %8.4f" % (process_time()-t0) )
        N = NE.npart
        Na = 3*N // 25
        rhofact = NE.lb / (NE.Lx * NE.Ly * NE.Lz)
        print( "# sides of the rectangular MD-box L = [ %8.4f %8.4f %8.4f ]" % (NE.Lx, NE.Ly, NE.Lz) )
        print( "# number of particles  N = %d" % N)
        print( "# density rho = %8.4f   (%8.4f)" % (NE.rho, rho) )
        print( "# potential cut-off radius  rcut = %8.4f *sigma" % NE.rcut)
        print( "# lateral size of slicing Deltaz = %8.4f" % (NE.Lz / NE.lb))
        # initial equilibrated condition
        t0 = process_time()
        estep = read_input(NE, N, conf_in=dir+"conf_eg0984.b")
        print("# initial conditions, cpu = %8.4f" % (process_time()-t0) )
        # Non-Equilibrium run with hot thermostat region
        t0 = process_time()
        pas= 0
        #dumpxyz(NE, N, Na, pas, dumpf)
        (enep, enek, etot, heat0, heat1, vcmx, vcmy, vcmz) = NE.nemdi( N, Na, mx, my, mz, kt, dkt, pas, lther )
        print("# starting dnemd trajectory\n")
        print( "    'pas'   'enep'     'enek'   'enet'      'heatin - heatout'   'vcm' ")
        print("%8.2f  %9.4f %9.4f %12.7f %8.3f %8.3f %12.2g %7.2g %7.2g" % (0.,enep, enek, etot, heat0, heat1, vcmx, vcmy, vcmz) )
        for it in range(nstep):
            (t, enep, enek, etot, heat0, heat1, vcmx, vcmy, vcmz) = NE.nemdr( N, Na, mx, my, mz, kt, dkt, pas, dt, freq, lther )
            print("%8.2f  %9.4f %9.4f %12.7f %8.3f %8.3f %12.2g %7.2g %7.2g" % (t,enep, enek, etot, heat0, heat1, vcmx, vcmy, vcmz) )
            pas += freq
            # writing 
            #dumpxyz(NE, N, Na, pas, dumpf)
    print( "# Non-Equilibrium run, cpu = %8.4f" % (process_time()-t0) )
    g=3*N-3
    nstep *= freq
    print( "# ending ne-md trajectory  <ep>=%6.1f  <ek>=%7.4f   T=%8.3f  P=%8.3f\n" % ( NE.ept/nstep, NE.ekt/nstep, 2.*NE.ekt/(nstep*g), NE.pres/nstep) )
    # final positions and momenta to file conf_in.b
    write_input(NE, N, nstep, conf_out=dir+"conf_ng0984.b")
    # saving with pickle
    #with open(pfile, 'wb') as ftrj:
    #    rhofact = NE.lb / (NE.Lx * NE.Ly * NE.Lz)
    #    dump( (dkt,rhofact,id) , ftrj, HIGHEST_PROTOCOL)
    #    dump( (NE.heatt,NE.hstat,NE.hstbt,NE.enet,NE.enkat,NE.enkbt,NE.jmazt,NE.gzt,NE.jezt), ftrj, HIGHEST_PROTOCOL)


### second part of trajectory

In [ ]:
if __name__ == "__main__":
    dir="./RUN/"
    pfile = dir + "ngmdB.pickle"
# r-parameters  # defaults
    nstep= 1000  #  number of multiples of 'freq' time steps 
    freq = 200  #  2000
    print( "# number of boxes mx = %d, my = %d, mz = %d" % (mx, my, mz) )
    print( "# base (kinetic) temperature kt = %8.4f  delta kt= %8.4f" % (kt, dkt) )
    print( "# integration time step dt = %8.4f" % dt )
    print( "# number of time steps for this run nst = %d " % (nstep) )    
    with open(dir+"trajectoryBg.xyz", 'w') as dumpf:
        t0 = process_time()
        NE = LJ(rho, mx, my, mz, gforce, nstep*freq)
        print("# creating NE object, cpu = %8.4f" % (process_time()-t0) )
        N = NE.npart
        Na = 3*N // 25
        rhofact = NE.lb / (NE.Lx * NE.Ly * NE.Lz)
        print( "# sides of the rectangular MD-box L = [ %8.4f %8.4f %8.4f ]" % (NE.Lx, NE.Ly, NE.Lz) )
        print( "# number of particles  N = %d" % N)
        print( "# density rho = %8.4f   (%8.4f)" % (NE.rho, rho) )
        print( "# potential cut-off radius  rcut = %8.4f *sigma" % NE.rcut)
        print( "# lateral size of slicing Deltaz = %8.4f" % (NE.Lz / NE.lb))
        # initial equilibrated condition
        t0 = process_time()
        estep = read_input(NE, N, conf_in=dir+"conf_ng0984.b")
        print("# initial conditions, cpu = %8.4f" % (process_time()-t0) )
        # Non-Equilibrium run with temperature gradient
        t0 = process_time()
        pas= 0
        #dumpxyz(NE, N, Na, pas, dumpf)
        (enep, enek, etot, heat0, heat1, vcmx, vcmy, vcmz) = NE.nemdi( N, Na, mx, my, mz, kt, dkt, pas, lther )
        print("# starting dnemd trajectory\n")
        print( "    'pas'   'enep'     'enek'   'enet'      'heatin - heatout'   'vcm' ")
        print(" %10.1f %9.4f %9.4f %12.7f %8.3f %8.3f %12.2g %7.2g %7.2g" % (0.,enep, enek, etot, heat0, heat1, vcmx, vcmy, vcmz) )
        for it in range(nstep):
            (t, enep, enek, etot, heat0, heat1, vcmx, vcmy, vcmz) = NE.nemdr( N, Na, mx, my, mz, kt, dkt, pas, dt, freq, lther )
            print(" %10.1f %9.4f %9.4f %12.7f %8.3f %8.3f %12.2g %7.2g %7.2g" % (t,enep, enek, etot, heat0, heat1, vcmx, vcmy, vcmz) )
            pas += freq
            # writing 
            #dumpxyz(NE, N, Na, pas, dumpf)
    print( "# Non-Equilibrium run, cpu = %8.4f" % (process_time()-t0) )
    g=3*N-3
    nstep *= freq
    print( "# ending ne-md trajectory  <ep>=%6.1f  <ek>=%7.4f   T=%8.3f  P=%8.3f\n" % ( NE.ept/nstep, NE.ekt/nstep, 2.*NE.ekt/(nstep*g), NE.pres/nstep) )
    # final positions and momenta to file conf_in.b
    write_input(NE, N, nstep, conf_out=dir+"conf_ng0984.b")
    # saving with pickle
    #with open(pfile, 'wb') as ftrj:
    #    rhofact = NE.lb / (NE.Lx * NE.Ly * NE.Lz)
    #    dump( (dkt,rhofact,id) , ftrj, HIGHEST_PROTOCOL)
    #    dump( (NE.heatt,NE.hstat,NE.hstbt,NE.enet,NE.enkat,NE.enkbt,NE.jmazt,NE.gzt,NE.jezt), ftrj, HIGHEST_PROTOCOL)
